<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Hierarchical Parameter Server Demo

## Overview

In HugeCTR version 3.5, we provide Python APIs for embedding table lookup with [HugeCTR Hierarchical Parameter Server (HPS)](https://nvidia-merlin.github.io/HugeCTR/master/hugectr_core_features.html#hierarchical-parameter-server)
HPS supports different database backends and GPU embedding caches.

This notebook demonstrates how to use HPS with HugeCTR Python APIs. Without loss of generality, the HPS APIs are utilized together with the ONNX Runtime APIs to create an ensemble inference model, where HPS is responsible for embedding table lookup while the ONNX model takes charge of feed forward of dense neural networks.

## Installation

### Get HugeCTR from NGC

The HugeCTR Python module is preinstalled in the 22.05 and later [Merlin Training Container](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-training): `nvcr.io/nvidia/merlin/merlin-training:22.05`.

You can check the existence of required libraries by running the following Python code after launching this container.

```bash
$ python3 -c "import hugectr"
```

**Note**: This Python module contains both training APIs and offline inference APIs. For online inference with Triton, please refer to [HugeCTR Backend](https://github.com/triton-inference-server/hugectr_backend).

> If you prefer to build HugeCTR from the source code instead of using the NGC container, please refer to the
> [How to Start Your Development](https://nvidia-merlin.github.io/HugeCTR/master/hugectr_contributor_guide.html#how-to-start-your-development)
> documentation.

## Data Generation

HugeCTR provides a tool to generate synthetic datasets. The [Data Generator](https://nvidia-merlin.github.io/HugeCTR/master/api/python_interface.html#data-generator-api) is capable of generating datasets of different file formats and different distributions. We will generate one-hot Parquet datasets with power-law distribution for this notebook:

## Train from Scratch

We can train fom scratch by performing the following steps with Python APIs:

1. Create the solver, reader and optimizer, then initialize the model.
2. Construct the model graph by adding input, sparse embedding and dense layers in order.
3. Compile the model and have an overview of the model graph.
4. Dump the model graph to the JSON file.
5. Fit the model, save the model weights and optimizer states implicitly.
6. Dump one batch of evaluation results to files.

## Convert HugeCTR to ONNX

We will convert the saved HugeCTR models to ONNX using the HugeCTR to ONNX Converter. For more information about the converter, refer to the README in the [onnx_converter](https://github.com/NVIDIA-Merlin/HugeCTR/tree/master/onnx_converter) directory of the repository.

For the sake of double checking the correctness, we will investigate both cases of conversion depending on whether or not to convert the sparse embedding models.

In [1]:
!pip3 install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 101.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 108.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 121.9 MB/s eta 0:00:00
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [1]:
from hugectr.inference import HPS, ParameterServerConfig, InferenceParams, VolatileDatabaseParams, PersistentDatabaseParams
import hugectr
import pandas as pd
import numpy as np

import onnxruntime as ort

import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

slot_size_array = [10000, 10000, 10000, 10000]
key_offset = np.insert(np.cumsum(slot_size_array), 0, 0)[:-1]
batch_size = 64

# 1. Configure the HPS hyperparameters
ps_config = ParameterServerConfig(
           emb_table_name = {"hps_demo": ["sparse_embedding1", "sparse_embedding2"]},
           embedding_vec_size = {"hps_demo": [128, 128]},
           max_feature_num_per_sample_per_emb_table = {"hps_demo": [2, 2]},
           volatile_db = VolatileDatabaseParams(
                type = hugectr.DatabaseType_t.redis_cluster,
                address =  "127.0.0.1:7000,127.0.0.1:7001,127.0.0.1:7002",
                user_name = "default",
                password = "",
                num_partitions = 8,
                max_get_batch_size = 100000,
                max_set_batch_size = 100000,
                overflow_margin = 10000000,
                overflow_resolution_target = 0.8,
                initial_cache_rate = 1.0,
                update_filters = [ ".+" ]),
            persistent_db = PersistentDatabaseParams(
                path = "/data/rocksdb",
                num_threads = 16,
                read_only = False,
                max_get_batch_size = 1,
                max_set_batch_size = 10000,
            ),
           inference_params_array = [
              InferenceParams(
                model_name = "hps_demo",
                max_batchsize = batch_size,
                hit_rate_threshold = 1.0,
                dense_model_file = "",
                sparse_model_files = ["sequential.model", "sequential.model"],
                deployed_devices = [0,1],
                use_gpu_embedding_cache = True,
                cache_size_percentage = 0.5,
                i64_input_key = True)
           ])

# 2. Initialize the HPS object
hps = HPS(ps_config)


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[HCTR][19:58:03.361][WARNING][RK0][main]: default_value_for_each_table.size() is not equal to the number of embedding tables
[HCTR][19:58:03.362][INFO][RK0][main]: Creating RedisCluster backend...
[HCTR][19:58:03.363][INFO][RK0][main]: RedisCluster: Connecting via 127.0.0.1:7000...
[HCTR][19:58:03.363][INFO][RK0][main]: Volatile DB: initial cache rate = 1
[HCTR][19:58:03.363][INFO][RK0][main]: Volatile DB: cache missed embeddings = 0
[HCTR][19:58:03.368][DEBUG][RK0][main]: RedisCluster backend. Table: hps_et.hps_demo.sparse_embedding1. Inserted 1000 / 1000 pairs.
[HCTR][19:58:03.369][INFO][RK0][main]: Table: hps_et.hps_demo.sparse_embedding1; cached 1000 / 1000 embeddings in volatile database (RedisCluster); load: 1000 / 80000000 (0.00%).
[HCTR][19:58:03.371][DEBUG][RK0][main]: RedisCluster backend. Table: hps_et.hps_demo.sparse_embedding2. Inserted 1000 / 1000 pairs.
[HCTR][19:58:03.372][INFO][RK0][main]: Table: hps_et.hps_demo.sparse_embedding2; cached 1000 / 1000 embeddings in volat

In [2]:
# 4. Make inference from the HPS object and the ONNX inference session of `hps_demo_without_embedding.onnx`.

key1 = np.arange(0, batch_size, dtype=np.ulonglong)
key2 =  np.arange(batch_size, batch_size * 2, dtype=np.ulonglong)

embedding1 = torch.zeros(batch_size * 128).to(device)
embedding2 = torch.zeros(batch_size * 128).to(device)
embd1_ptr = embedding1.data_ptr()
embd2_ptr = embedding2.data_ptr()

print("{:x}".format(embd1_ptr))
print(type(embd1_ptr))
print("{:x}".format(embd2_ptr))
print(type(embd2_ptr))

hps.lookup(key1, "hps_demo", 0,embd1_ptr,0)
hps.lookup(key2, "hps_demo", 1,embd2_ptr,0)
embedding1 = embedding1.reshape(batch_size, 128)
embedding2 = embedding2.reshape(batch_size, 128)

print(embedding1)
print(embedding2)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")


key3 = np.arange(0, batch_size, dtype=np.ulonglong)
key4 =  np.arange(batch_size, batch_size * 2, dtype=np.ulonglong)

embedding3 = torch.zeros(batch_size * 128).to(device)
embedding4 = torch.zeros(batch_size * 128).to(device)
embd3_ptr = embedding3.data_ptr()
embd4_ptr = embedding4.data_ptr()

hps.lookup(key3, "hps_demo", 0,embd3_ptr,1)
hps.lookup(key4, "hps_demo", 1,embd4_ptr,1)
embedding3 = embedding3.reshape(batch_size, 128)
embedding4 = embedding4.reshape(batch_size, 128)

print(embedding3)
print(embedding4)


7f1e9a400000
<class 'int'>
7f1e9a408000
<class 'int'>
[HCTR][19:58:12.083][DEBUG][RK0][main]: RedisCluster backend. Table: hps_et.hps_demo.sparse_embedding1. Fetched 64 / 64 values.
[HCTR][19:58:12.084][DEBUG][RK0][main]: RedisCluster backend. Table: hps_et.hps_demo.sparse_embedding2. Fetched 64 / 64 values.
tensor([[0.0000e+00, 1.0000e+00, 2.0000e+00,  ..., 1.2500e+02, 1.2600e+02,
         1.2700e+02],
        [1.2800e+02, 1.2900e+02, 1.3000e+02,  ..., 2.5300e+02, 2.5400e+02,
         2.5500e+02],
        [2.5600e+02, 2.5700e+02, 2.5800e+02,  ..., 3.8100e+02, 3.8200e+02,
         3.8300e+02],
        ...,
        [7.8080e+03, 7.8090e+03, 7.8100e+03,  ..., 7.9330e+03, 7.9340e+03,
         7.9350e+03],
        [7.9360e+03, 7.9370e+03, 7.9380e+03,  ..., 8.0610e+03, 8.0620e+03,
         8.0630e+03],
        [8.0640e+03, 8.0650e+03, 8.0660e+03,  ..., 8.1890e+03, 8.1900e+03,
         8.1910e+03]], device='cuda:0')
tensor([[ 8192.,  8193.,  8194.,  ...,  8317.,  8318.,  8319.],
        [ 832